In [ ]:
import serial
import time
from traitlets import HasTraits, List
import numpy as np

In [ ]:
#Open a serial connection with the Arduino Mega
ser = serial.Serial('COM3', 115200)

In [1]:
#Create an encoder class with traits
class Encoders(HasTraits):    
    encoderValues = List() #We store the left and right encoder value in a list
    
    def __init__(self, encoderValues, deltaTicks):
        self.encoderValues = encoderValues        
        self.deltaTicks = deltaTicks       

#Create an encoder instance 
encoders = Encoders([0,0], [0,0])

#Create a function that is triggered when a change to encoders is detected
def monitorEncoders(change):
        
        if change['new']:
            #Message for debugging only         
            print("Success!")
            
encoders.observe(monitorEncoders, names = "encoderValues")

NameError: name 'HasTraits' is not defined

In [ ]:
#Create a move function that sends move commands to the Arduino
def move(linearVelocity, angularVelocity):       
        
        command = f"<{linearVelocity},{angularVelocity}>"
        ser.write(str.encode(command))       

In [ ]:
#Functions to read and format encoder data received from the Serial port
def formatData(data):
    delimiter = "x"
    leftVal = ""
    rightVal = ""
    
    for i in range(len(data)):        
        if data[i] == ",":
            delimiter = ","            
        elif delimiter != "," and data[i].isdigit():
            leftVal += data[i]            
        elif delimiter == "," and data[i].isdigit():
            rightVal += data[i]    
       
    leftVal, rightVal = int(leftVal), int(rightVal)
    encoders.encoderValues = [leftVal, rightVal]    
    print("Encoders: ", encoders.encoderValues)

def handleSerial():
    #ser.readline() waits for the next line of encoder data, which is sent by Arduino every 50 ms
    temp = ser.readline()
    data = temp.decode()
    formatData(data)  

In [ ]:
#Change this command for testing
move(0.0,0.0)
handleSerial()

In [ ]:
ser.close()